In [1]:
import pandas as pd
import numpy as np
import os
from sqlalchemy import create_engine
import sys
os.getcwd()
sys.path.append("C:/SQL_PW/")
import config 

path = "//Resource//"
file = os.path.join("Resource", "Food_Access_Research_Atlas.csv")
foodaccess_df=pd.read_csv(file)
foodaccess_df.head()


,CensusTract,State,County,LILATracts_1And10,LILATracts_halfAnd10,LILATracts_1And20,LILATracts_Vehicle,Urban,Rural,LA1and10,...,lapop20,lapop20share,lalowi20,lalowi20share,lakids20,lakids20share,laseniors20,laseniors20share,lahunv20,lahunv20share
0,1001020100,AL,Autauga,0,0,0,0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1001020200,AL,Autauga,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1001020300,AL,Autauga,0,0,0,0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1001020400,AL,Autauga,0,0,0,0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1001020500,AL,Autauga,0,0,0,0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
##[Introduction]In this data,several indicators are available to measure food access along these dimensions./
            #So, we can use it to explain the phenomenons that found by my teammates in their data that related to the Food Stamp

##we can use follwing code to filter the state if is needed
#foodaccess_df_NC=foodaccess_df[foodaccess_df["State"]=="NC"]


## Get  separate Urban and Rural data where there is no Food Quarter in the region 
foodaccess_urban=foodaccess_df[(foodaccess_df["Urban"]==1) & (foodaccess_df["NUMGQTRS"]==0)]
foodaccess_rural=foodaccess_df[(foodaccess_df["Rural"]==1) & (foodaccess_df["NUMGQTRS"]==0)]


##[Analysis] To see if  there is a correlation between the factor of low access / 
            #& the ratio of stores with food stamp(Dan's data) or the individual paticipants with foor stamp(Zhen's data)

#[Urban]Get areas with low access 
data_urban_la=foodaccess_urban[["CensusTract", "State", "County", "LATracts_half","LATracts1"]]

#[Rural]Get areas with low access
data_rural_la=foodaccess_rural[["CensusTract", "State", "County", "LATracts10","LATracts20"]]


##[Analysis] the low vehicle access(the households that do not have vehicle ) 
            #To see how retaed the vehicle access to low access and the the individual paticipants with foor stamp(Zhen's data)

#[Urban]what percent of households do not have vehicle measured at 1 mile
data_urban_vehicle=foodaccess_urban[["CensusTract", "State", "County", "lahunv1share"]]

#[Rural]what percent of households do not have vehicle measured at 20 mile
data_rural_vehicle=foodaccess_rural[["CensusTract", "State", "County", "lahunv20share"]]

##[Analysis] the ratio of food stamp paticipants(Zhen's data) in total poplation(Bing's data)

#get the population data from the database 
data_pop=foodaccess_df[["CensusTract", "State", "County", "POP2010"]]



In [4]:
pw = config.password
connection_string = ("root:" + pw + "@localhost/snap_db")
engine = create_engine(f'mysql://{connection_string}')

In [13]:
engine.table_names()

['data_rural_la',
 'data_rural_vehicle',
 'data_urban_la',
 'data_urban_vehicle',
 'foodaccess_rural',
 'foodaccess_urban',
 'population']

In [12]:
foodaccess_urban.to_sql(name='foodaccess_urban', con=engine, if_exists='replace', index=False)
foodaccess_rural.to_sql(name='foodaccess_rural', con=engine, if_exists='replace', index=False)
data_urban_la.to_sql(name='data_urban_la', con=engine, if_exists='replace', index=False)
data_rural_la.to_sql(name='data_rural_la', con=engine, if_exists='replace', index=False)
data_urban_vehicle.to_sql(name='data_urban_vehicle', con=engine, if_exists='replace', index=False)
data_rural_vehicle.to_sql(name='data_rural_vehicle', con=engine, if_exists='replace', index=False)
data_pop.to_sql(name='population', con=engine, if_exists='replace', index=False)

In [ ]:
pd.read_sql_query('select * from FoodAccessResearch', con=engine).head(5)